# OSM Traces (GPX files)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anitagraser/movingpandas-examples/main?filepath=2-analysis-examples/5-osm-traces.ipynb)

<img align="right" src="https://anitagraser.github.io/movingpandas/pics/movingpandas.png">

This notebook illustrates the use of GPS traces shared publicly by OSM community members in GPX format. 

Source: https://www.openstreetmap.org/traces

In [ ]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd

from os.path import exists
from urllib.request import urlretrieve
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta

import warnings
warnings.simplefilter("ignore")

In [ ]:
mpd.__version__

## Download OSM traces and generate a GeoDataFrame

In [ ]:
def get_osm_traces(page=0, bbox='16.18,48.09,16.61,48.32'):
    file = 'osm_traces.gpx'
    url = f'https://api.openstreetmap.org/api/0.6/trackpoints?bbox={bbox}&page={page}'
    if not exists(file):
        urlretrieve(url, file)
    gdf = gpd.read_file(file, layer='track_points')
    # OPTIONAL: dropping empty columns
    gdf.drop(columns=['ele', 'course', 'speed', 'magvar', 'geoidheight', 'name', 'cmt', 'desc',
       'src', 'url', 'urlname', 'sym', 'type', 'fix', 'sat', 'hdop', 'vdop',
       'pdop', 'ageofdgpsdata', 'dgpsid'], inplace=True) 
    return gdf

## TrajectoryCollection from OSM traces GeoDataFrame

In [ ]:
gdf = get_osm_traces()
osm_traces = mpd.TrajectoryCollection(gdf, 'track_fid', t='time')
print(f'The OSM traces download contains {len(osm_traces)} tracks')

In [ ]:
for track in osm_traces: print(f'Track {track.id}: length={track.get_length():.0f}m')

In [ ]:
track.plot()

## Generalizing and visualizing

Generalization is optional but speeds up rendering

In [ ]:
osm_traces = mpd.MinTimeDeltaGeneralizer(osm_traces).generalize(tolerance=timedelta(minutes=1))
osm_traces.hvplot(title='OSM Traces', line_width=7, width=700, height=400)

In [ ]:
osm_traces.get_trajectory(0).hvplot(title='Speed (m/s) along track', c='speed', cmap='RdYlBu',
                                    line_width=7, width=700, height=400, tiles='CartoLight', colorbar=True)

## Continue exploring MovingPandas

1. [Bird migration analysis](1-bird-migration.ipynb)
1. [Ship data analysis](2-ship-data.ipynb)
1. [Horse collar data exploration](3-horse-collar.ipynb)
1. [Stop hotspot detection](4-stop-hotspots.ipynb)
1. [OSM traces](5-osm-traces.ipynb)